In [ ]:
import numpy as np
import scipy.linalg
from tools.L96_model import lorenz96, Lorenz96_and_TLM
from tools.enkf import kfs
from tools.plots import plotL96, plotTimeseries, plotTimeseriesWithObs, tileplotB
from tools.plots import plot_LocMatrix, compare_schemes, compare_RMSE, compareB, compare_covariances
from tools.obs import createH, gen_obs
from tools.cov import getBClimate, getPbs, evolve_cov, evolve_ensemble_cov
from tools.enkf import getlocmat

from tools.var3d_cvt import var3d
from tools.var4d_cvt import var4d
from tools.En4DVar import En4DVar
from tools.L4DEnVar import L4DEnVar

# Practical: Experiments with hybrid data assimilation methods

In this practical, you will explore hybrid DA methods in the Lorenz 1996 model. The hybrid DA methods take advantage of the variational methods as well as the ensemble methods allowing for flow-dependent background error covariance matrices.

You will explore 2 types of hybrid methods:
1. Those that use a hybrid background error covariance in a traditional variational minimisation framework.
2. Those that avoid computing tangent linear and adjoint models and instead use 4-dimensional (cross-time) ensemble covariances to communicate the impact of observations to the initital time. Here, you will use SC-4DEnsemble Var (SC denotes strong constraint).

As hybrid DA introduces the ensemble into the variational method, this practical will help you revise the pros and cons of the ensemble Kalman filter as well.

## Setting up the experiment
Similar to previous practicals, we will set up a twin experiment. In the twin experiment, we will have a nature run as our modelled truth. Observations will be sampled based on the modelled truth and our chosen observation operator. We will also setup a background error covariance matrix for the variational method.

# Nature run
Here, similar to previous practicals, we generate a nature run as the modelled truth. Recall that the Lorenz 1996 model is
\begin{equation}
\frac{\partial x_{n}}{\partial t} = (x_{n+1}-x_{n-2})x_{n-1} - x_{n} + F
\end{equation}
for $n = 1, \dots, N_{x}$ with $N_{x} \geq 4$. Variables $x_{n}$ are assumed to be periodic, e.g. $x_{N_{x}}=x_{1}$. 

In this practical, we choose `12` variables with a forcing, `F = 8.0`.

In [ ]:
Nx = 12       #  No. of state variables (No. of nodes in Lorenz-96, often n=40)
tmax = 14.    # The final time of the simulation
discard = 20  # time step for spin up 
dt = 0.025    # Time step of integration scheme.
F = 8.0       # Forcing of L96
# create initial condition
x0 = F*np.ones(Nx)
x0[0] = x0[0] + 5

#Create the truth.
t = np.arange(0, tmax+dt, dt)
xt = lorenz96(x0, tmax, dt, discard, F)

Nsteps = len(t)
print("Nsteps", Nsteps)

# Plot time series for each model variable
ncols = 3  # Number of columns in subplots
plotTimeseries(Nx, t, xt, ncols, linewidth=1)

# Plot Hovmöller diagram (time against space)
plotL96(t, xt, Nx, 'Lorenz 96 system (truth)')

### Generate synthetic observations

Various observation networks can be used to generate observations based on the modelled truth. You can choose the following observation grid:
* all: this chooses $\mathbf{H}=\mathbf{I}$. It means that there are
$n$ direct observations (at each time observations are asked for
-- see below).
* 1010: this specifies that direct observations are made at every other
grid point. There are $n/2$ observations at each observation time.
* landsea: this simulates the effect of observing directly only half
of the domain (in reality there are many more observations over land
than over sea). This option assigns half of the grid points to 'land'
and the other half as 'sea', where there are observation over land
only. There are $n/2$ observations at each observation time.


In [ ]:
period_obs = 2   # number of time steps between observations
obsgrid = "1010" #'all','1010', or 'landsea'
footprint = 3
var_obs    = 1.0    # observation error variance
seed       = 1      # random number seed

# Create the observation operator matrix
p, H = createH(obsgrid, Nx, footprint)
# Generate synthetic observations, ob err cov matrix, and return their validity times
tobs, y, R = gen_obs(t, xt, period_obs, H, var_obs, seed)
# for foot_6, each model variable corresponds to multiple observations
# the plot shows the averaged observations that depend on the model variable.
plotTimeseriesWithObs(Nx, t, xt, tobs, y, H, ncols, linewidth=1)

### Generate climatological background error covariances

Hybrid methods take advantage of both variational methods and ensemble methods. In variational methods, the uncertainty of the background state is provided by the background error covariance  ($\mathbf{B}$) matrix.

In previous practicals, we used Canadian Quick method to estimate the $\mathbf{B}$ matrix for variational methods. In this practical, instead of using Canadian Quick, we specify a climatological $\mathbf{B}$ matrix where the error correlations decrease gradually with distance.

In [ ]:
Bc = getBClimate(Nx)
# Plot background error covariance matrix
mycmap      = 'BrBG'  
vs          = [-np.amax(np.diag(Bc)),np.amax(np.diag(Bc))]     
tileplotB(Bc, mycmap, vs)

## Variational method with control variable transforms

The 3D-Var cost function can be written in an incremental formulation:
\begin{equation}
J^{3\mathrm{D}}(\mathbf{\delta x})=\frac{1}{2}\mathbf{\delta x}^\mathrm{T}\mathbf{B}^{-1}\mathbf{\delta x}+\frac{1}{2}\left(\mathbf{y}-\mathcal{H}(\mathbf{x}^{\mathrm{b}})-\mathbf{H}\mathbf{\delta x}\right)^{\mathrm{T}}\mathbf{R}^{-1}\left(\vphantom{\mathbf{x}_{i}^{\mathrm{R}}}\bullet\right),
\end{equation}
where $\mathbf{\delta x}$ is the increment.

Some formulations of hybrid data assimilation methods take advantage of the control variable transforms (CVTs). In this practical, the control variable $\mathbf{\delta x} = \mathbf{U}\mathbf{v}_B$ is used to model the background error covariance matrix, $\mathbf{B} = <\mathbf{\delta x} \mathbf{\delta x}^\mathrm{T}>$. 

In a pure variational method, the $\mathbf{U} = \mathbf{B}^{\frac{1}{2}}$, and the cost function becomes:
\begin{equation}
J^{3\mathrm{D}}(\mathbf{v}_B)=\frac{1}{2}\mathbf{v}_B^\mathrm{T}\mathbf{v}_B+\frac{1}{2}\left(\mathbf{y}-\mathcal{H}(\mathbf{x}^{\mathrm{b}})-\mathbf{H}\mathbf{B}^{\frac{1}{2}}\mathbf{v}_B\right)^{\mathrm{T}}\mathbf{R}^{-1}\left(\vphantom{\mathbf{x}_{i}^{\mathrm{R}}}\bullet\right).
\end{equation}

Similarly, we can formulate the strong-constraint 4DVar cost function:
\begin{equation}
J^{4\mathrm{D}}(\mathbf{v}_B)=\frac{1}{2}\mathbf{v}_B^\mathrm{T}\mathbf{v}_B+\frac{1}{2}\sum_{i=0}^{T}\left(\mathbf{y}_{i}-\mathcal{H}_{i}(\mathbf{x}_{i}^{\mathrm{b}})-\mathbf{H}_{i}\mathbf{M}_{i-1}\mathbf{M}_{i-2}\ldots\mathbf{M}_{0}\mathbf{B}^{\frac{1}{2}}\mathbf{v}_B\right)^{\mathrm{T}}\mathbf{R}_{i}^{-1}\left(\vphantom{\mathbf{x}_{i}^{\mathrm{b}}}\bullet\right).
\end{equation}

Here, we run both 3DVar and strong-constraint 4Dvar experiments in a CVTs formulation. What can you say about the performance of these two methods?

In [ ]:
ug0 = 0.5*(xt[:, 10] + xt[:, 40]) # generate initial guess of the background
invR = np.linalg.inv(R)           # inverse of the observation error covariance matrix
B0sq = scipy.linalg.sqrtm(Bc)     # square root of the B matrix (U matrix)
ub3, ua3 = var3d(ug0, t, tobs, y, H, B0sq, invR, F)
# 4DVar
anawin = 2 # number of observations per window
# ub4 is the background and ua4 is the analysis from the 4DVar
ub4, ua4 = var4d(ug0, t, period_obs, anawin, y, H, B0sq, invR, F)

# plotting the comparison of backgrounds, analyses and the truth and their RMSEs
compare_schemes(Nx, t, xt, ncols, [ub3, ua3, ub4, ua4], labels=["Truth", "bgd 3DV", "ana 3DV", "bgd 4DV", "ana 4DV"])
compare_RMSE(Nsteps, t, xt, [ub3, ua3, ub4, ua4], H, 
             ["bgd 3DV", "ana 3DV", "bgd 4DV", "ana 4DV"], lab_cols=4)

## Local ensemble transform Kalman filter

In the ensemble Kalman filter, the forecast error covariance matrix is estimated by an ensemble of model realisations. If $x^{b,(n)}$ is the model state in the $n$th ensemble member before analysis, the $\mathbf{P}^f$ matrix is
\begin{equation}
\mathbf{P}^f = \frac{1}{N_{ens}-1} \sum_{n=1}^{N_{ens}} (x^{b,(n)}-\overline{x^{b}})
(x^{b,(n)}-\overline{x^{b}})^{\rm{T}}
\end{equation}

and $\overline{x^{b}} = \frac{1}{N_{ens}}\sum_{n=1}^{N_{ens}} x^{b,(n)}$ the forecast ensemble mean. The $\mathbf{P}^f$ matrix estimated from an ensemble can have suprious correlations in space due to under sampling, and localisation is used to remove the spurious spatial correlations. 

In the state space, the $\mathbf{P}^f$ matrix can be tapered by a localisation matrix $\mathbf{L}$ using the Schur product (element-wise matrix product), $\mathbf{L} \circ \mathbf{P}^f$. This is called $\mathbf{P}^f$ localisation. The localisation matrix gradually decreases the correlation in space. 

Localisation can also be done in the observation space. This approach is sometimes called domain localisation as well. To understand this approach, it is worth assuming that only one grid point is in the state vector (We will provide a bit more details later). In this approach, the localisation matrix is used to taper the $\mathbf{R}$ matrix such that the observation errors are inflated when they are away from the model grid point. Under this condition, the observations far away from the grid point has little impact on the analysis. This is also called $\mathbf{R}$ localisation. 

Here, to look at the effect of localisation, we can play with the localisation matrix $\mathbf{L}$. When `loctype='cutoff'`, the localisation matrix follows a cut-off function and when `loctype='GC'`, the matrix follows the Gaspari-Cohn function.
Try

<table>
 <thead>
   <tr>
     <th>loctype</th>
     <th>lam</th>
   </tr>
 </thead>
 <tbody>
   <tr>
     <td>cutoff</td>
     <td>0.1</td>
   </tr>
   <tr>
     <td>cutoff</td>
     <td>2</td>
   </tr>
   <tr>
     <td>cutoff</td>
     <td>10</td>
   </tr>
   <tr>
     <td>GC</td>
     <td>0.1</td>
   </tr>
   <tr>
     <td>GC</td>
     <td>2</td>
   </tr>
   <tr>
     <td>GC</td>
     <td>10</td>
   </tr>
 </tbody>
</table>

Can you interpret these plots? What is the difference between the cut-off function and the Gaspari-Cohn function, and different `lam`?

In [ ]:
lam = 2.
# localisation halfwidth
loctype ='GC'  # Gaspari-Cohn
Lxx = getlocmat(Nx, Nx, np.eye(Nx), lam, loctype)  # get the localisation matrix
Lxy = getlocmat(Nx, p, H, lam, loctype)  # get the localisation matrix

# Plot background error covariance matrix
mycmap      = 'BrBG'  
tileplotB(Lxx, mycmap, [0, 1], title="Localization in model space")
tileplotB(Lxy, mycmap, [0, 1], title="Localization in model/obs space")

For now, let's settle for: `loctype = 'GC'` and `lam = 2`. The implementation of domain localisation depends on the ensemble transform Kalman filter (ETKF). In the ETKF, assume $\mathbf{X}^f$ is a matrix of ensemble anomaly with each column being the anomaly for each member, the analysis increment can be expressed as 
$$\mathbf{\delta x} = \mathbf{X}^f\mathbf{Y}^\mathrm{T}\left(\mathbf{Y}\mathbf{Y}^\mathrm{T} + (N_{ens}-1)\mathbf{R}\right)^{-1}\left(\mathbf{y} - \mathcal{H}\left(\mathbf{x}^f\right)\right) = \mathbf{X}^f\mathbf{w},$$
while the ensemble analysis is updated by a transformation matrix $\mathbf{T}$, leading to an update equation, 
$$\mathbf{X}^a = \mathbf{X}^f\mathbf{T}.$$

The terms $\mathbf{w \in \mathbb{R}^{N_{ens}}}$ and $\mathbf{T} \in \mathbb{R}^{N_{ens} \times N_{ens}}$ are calculated in observation space. Therefore, each model grid point (element in the state vector) can be updated independently using a subset of the total observations that are within a certain distance from that grid point. We can do this for all grid points sequentially or in parallel. This is the core idea of local ensemble transform Kalman filter (LETKF). In this case, the $\mathbf{R}$-localisation is usually used to taper the observation error covariance matrix.

The transform matrix in the above equation is given as $\mathbf{T}\mathbf{T}^\mathrm{T} = \left[\mathbf{I} + \frac{1}{N_{ens} - 1}\mathbf{Y}^\mathrm{T}\mathbf{R}^{-1}\mathbf{Y}\right]^{-1}$. With a eigenvalue decomposition, we can express $\mathbf{T} = \mathbf{U}\mathbf{\Sigma}^{-\frac{1}{2}}\mathbf{U}^\mathrm{T}$. The $i$-th column of $\mathbf{Y}$ is $col_i(\mathbf{Y}) = \mathcal{H}(\mathbf{x}^f)_i - \overline{\mathcal{H}(\mathbf{x}^f)}$. When using domain localisation, local $\mathbf{Y}$, $\mathbf{y}$ and $\mathbf{R}$ are formed by selecting the rows of global $\mathbf{Y}$, elements in global $\mathbf{y}$ and entries in global $\mathbf{R}$ that correspond to the chosen observations.

Perform DA assimilation using the LETKF. What can you say about the RMSE plots?

In [ ]:
M = 10  # ensemble size
met = 'ETKF' # method
lam = 2.   # localisation radius
loctype = 'GC'  # type of localisation function
# Ubkf is the forecast ensemble perturbations
# ubkf is the forecast ensemble mean
# Uakf is the analysis ensemble perturbation
# uakf is the analysis ensemble mean
Ubkf, ubkf, Uakf, uakf, _, _ =  kfs(ug0, F, lorenz96, t, tobs, y, H, R, 0.05, M, met,
        lam=lam, loctype=loctype, back0='random', desv=1.0) 

compare_schemes(Nx, t[:100], xt[:, :100], ncols, [ubkf[:, :100], uakf[:, :100]], labels=["Truth", "bgd mean", "ana mean"])
compare_RMSE(Nsteps, t, xt, [ub3, ua3, ub4, ua4, ubkf, uakf], H, 
             ["bgd 3DV", "ana 3DV", "bgd 4DV", "ana 4DV", "bgd LETKF", "ana LETKF"], lab_cols=3)

### Compare the climatological B with some sample B's
One of the purposes of hybrid DA is to combine covariance information from a static yet full-rank source (the climatological $\mathbf{B}$) used in the VAR methods, with the flow-dependent yet low-rank information coming from a sample of trajectories (the $\mathbf{P}^f$ obtained by ensemble methods).

Compare the climatological $\mathbf{B}$ with that obtained by the LETKF (computed from the background ensemble) at different times. The raw and localised versions are ploted for different times instants. In this case you can modify the variables `nsample`, which is the number of instants in which you want to display the $\mathbf{P}^f(t)$. How would this change if you increase or decrease the number of ensemble members? (hint: you need to rerun the LETKF to change the number of ensemble members.)


In [ ]:
nsample = 3 # number of time steps being shown for the forecast covariance matrix
Pbs_kf, LPbs_kf = getPbs(Lxx, Ubkf, Nx, nsample, period_obs)
# show 
compareB(Bc, Pbs_kf, LPbs_kf, nsample)

## Hybrid-En4DVar

Here, we test one hybrid DA approach where we run a strong constraint 4DVar system in parallel with LETKF. The LETKF system runs independently from the 4DVar but the 4DVar system uses information from the LETKF. The 4DVar system hybrids the static background error covariance matrix ($\mathbf{B}$) with the flow-dependent forecast error covariance matrix ($\mathbf{P}^f$) from LETKF. Therefore, the 4DVar system uses a flow-dependent error covariance matrix:
\begin{equation}
\mathbf{P}_h = \beta_0 \mathbf{B} + \beta_1 \mathbf{P}^f,
\end{equation}
where $\beta_0 + \beta_1 = 1$. The weighting between the static and the flow-dependent error covariance matrix can be case-dependent.

In this experiment, try changing the localisation radius (`lam`), ensmeble size (`ne`), observations per window (`anawin`) and the weigting for the static (first number) and the dynamic (second number) part of the covariance matrix (`beta`) and see how the results change.

In [ ]:
lam = 2.          # localisation radius
loctype = 'GC'       # type of localisation function
ne = 10              # number of ensemble members
anawin = 2           # number of observations per window
beta = [1., 0.]    # weighting for the B matrix (beta[0]), and the flow-dependent ETKF covariance (beta[1])

# the returned values are the background from Var (ubh), the analysis from Var (uah),
# the analysis of the ensemble anomaly from LETKF (Uaenh) and the analysis of the ensemble mean from LETKF (uaenh)  
ubh, uah, Uaenh, uaenh =  En4DVar(ug0, t, period_obs, anawin, ne, y, H, Bc, R, beta, F, rho=0.05, lam=lam, loctype='GC')


compare_schemes(Nx, t[100:200], xt[:, 100:200], ncols, [ubh[:, 100:200], uah[:, 100:200]], labels=["Truth", "bgd hybrid", "ana hybrid"])
compare_RMSE(Nsteps, t, xt, [ubh, uah], H, ["bgd 4DVar-LETKF", "ana 4DVar-LETKF"], lab_cols=2)

## 4DEnVar

In the 4DEnVar, the background error covariance matrix $\mathbf{B}$ is flow-dependent estimated in an ensemble Kalman filter fashion. The formulation of the 4DEnVar cost function takes advantage of the control variable transofmration of the 4DEnVar. Recall that the analysis increment can be written as $\mathbf{\delta x} = \mathbf{U}\mathbf{v}$. The 4DEnVar specifies that $\mathbf{U} = \mathbf{X}^f$ where the $i$-th column of the $col_i(\mathbf{X}^f) =  \frac{1}{\sqrt{N_{ens}-1}} (x^{f,(i)}-\overline{x^{f}})$, which is the anomaly of the $i$-th ensemble normalised by the square root of the ensemble size. 

Recall that in the pure variatonal method, $\mathbf{U} = \mathbf{B}^\frac{1}{2}$. In both the 4DEnVar and 4DVar, $\mathbf{B} = \mathbf{U}\mathbf{U}^\mathrm{T}$. Similar to the Hybrid En4DVar, 4DEnVar also enjoys the benefit of a flow-dependent error covariance matrix. The 4DEnVar further avaoids the use of tangent linear and adjoint models by using the ensemble forecast anomaly. The cost function is:
\begin{equation}
J^{4\mathrm{DEnVar}}(\mathbf{v}_{ens})=\frac{1}{2}\mathbf{v}_{ens}^\mathrm{T}\mathbf{v}_{ens}+\frac{1}{2}\sum_{i=0}^{T}
\left(
\mathbf{y}_{i}-\mathcal{H}_{i}(\mathbf{x}_{i}^{\mathrm{b}}) - \mathbf{H}_{i}\mathbf{X}^f_i\mathbf{v}_{ens}
\right)^{\mathrm{T}}
\mathbf{R}_{i}^{-1}\left(\vphantom{\mathbf{x}_{i}^{\mathrm{b}}}\bullet\right).
\end{equation}
where the $col_j(\mathbf{X}^f_i) = \mathcal{M}_{0 \rightarrow i}\left(\mathbf{x}^{f,(j)}_0\right) - \overline{\mathcal{M}_{0 \rightarrow i}\left(\mathbf{x}^{f, (j)}_0\right)}$.

### Exploring 4D background covariances

In the strong-constraint 4DVar, observations within the DA window are used to update the initial state of the model. This means that the initial condition is estimated based on observations from future time. Assume that the $\mathbf{v}_{ens}$ is obtained when $$\mathrm{grad}\mathbf{J}^\mathrm{4DEnVar} = \mathbf{v}_{ens} - \sum_{i=0}^{T}
\mathbf{X}_{i}^{f\mathrm{T}}\mathbf{H}^\mathrm{T}_{i}\mathbf{R}_{i}^{-1}\left(
\mathbf{y}_{i}-\mathcal{H}_{i}(\mathbf{x}_{i}^{\mathrm{b}}) - \mathbf{H}_{i}\mathbf{X}^f_i\mathbf{v}_{ens}
\right) = 0$$

Without actually solving the above equation, based on the CVT, we can see the the analysis increment is
$$\mathbf{\delta x}_0 = \mathbf{X}_0^f\mathbf{v}_{ens} = \sum_{i=0}^{T}
\mathbf{X}_0^f\mathbf{X}_{i}^{f\mathrm{T}}\mathbf{H}^\mathrm{T}_{i}\mathbf{R}_{i}^{-1}\left(
\mathbf{y}_{i}-\mathcal{H}_{i}(\mathbf{x}_{i}^{\mathrm{b}}) - \mathbf{H}_{i}\mathbf{X}^f_i\mathbf{v}_{ens}
\right)$$
Here, the term $\mathbf{X}_0^f\mathbf{X}_{i}^{f\mathrm{T}}$ is an error covariance between the initial time and $i$-th observation time in the DA window. Such a cross time error covariance matrix appears in the standard strong constraint 4DVar (SC-4DVar) as well where the SC-4DVar uses the adjoint model to compute the error covariance. Here, the 4DEnVar computes these error covariance matrices using an ensemble approximation. This means that the background error covariance of the 4DEnVar depends on how good the ensemble is. 

Before running the 4DEnvar, let us start by comparing the error covariance matrix: 
1. using tangent linear and adjoint models,
2. using an ensemble run with different initial conditions (sampled from a normal distribution centered on the truth with covariance Bc).

Try change the ensemble size (`ne`) and the time correlation between the initial time and different `lags`.
This cell will plot three rows of covariances. Can you tell what is being plotted in each row?

In [ ]:
nt = len(t) # number of total time steps
lags = 5  # lags between the first time step and the final time steps for cross time error covariance matrix
ne = 10  # number of ensemble members

# finding the tangent linear model at different time steps
_, TLM = Lorenz96_and_TLM(np.eye(Nx), nt, ug0, F, dt)
# get the covariance matrix (Bt) and time-correlated covariance matrix (B0t) by tangent linear model
Bt, B0t = evolve_cov(Bc, TLM, Nx, lags)
# get the covariance matrix (Pbt) and the time-correlated covariance matrix (Pb0t) from the ensemble
Pbt, Pb0t = evolve_ensemble_cov(x0, Nx, ne, lags, dt, F, B0sq)

clim = np.max(np.abs(np.diag(Bt[..., 0])))
compare_covariances(Bt, Pbt, Lxx, lags, clim, "RdBu", "")  # 3D covariances
compare_covariances(B0t, Pb0t, Lxx, lags, clim, "RdBu", "0,")  # 4D covariances

Finally, we will run the SC-4DEnVar. There is one aspect that we haven't mentioned is the generation of the ensemble in the 4DEnVar. There are multiple ways of doing this. In this practical, the following steps are performed for each DA window:
1. An LETKF is performed to generate the analysis ensemble perturbations/anomalies
2. A free ensemble run using the 4DEnVar analysis from the last window is performed to generate the forecast ensemble, which will be used to get $\mathbf{X}^f_i$.
3. A 4DEnVar is performed to obtain an analysis as ensemble mean, while the LETKF analysis ensemble anomaly is used to obtain a full analysis ensemble. 

Here, feel free to play with the localisation radius `lam`, the ensemble size `ne`, and switch on/off the localisation using `doLoc`. We use a fixed (in time) localisation. Remember this can be problematic when localising cross-time covariances in long assimilation windows. Can you think of a way to test this?

In [ ]:
anawin = 2 # number of observation per DA window
lam = 1.5 # localisation radius
loctype = 'GC' # type of localisation function
ne = 10  # number of ensemble members
doLoc = True # switch for localisation. If True, do localisation, else no localisation
rho = 0.05 # inflation factor

# x_a is the analysis from 4DEnVar
# x_b is the background for 4DEnVar
# VarEns is the ensemble analysis of the LETKF
# VarEns_ is the analysis ensemble mean of the LETKF
# FreeEns is the free ensemble run
x_a, x_b, VarEns, VarEns_, FreeEns = L4DEnVar(ug0, t, period_obs, anawin, ne, y, H, Bc, R, F, rho, doLoc, lam=lam, loctype=loctype)

compare_schemes(Nx, t, xt, ncols, [x_b, x_a], labels=["Truth", "bgd hybrid", "ana hybrid"])
compare_RMSE(Nsteps, t, xt, [x_b, x_a], H, ["bgd hybrid", "ana hybrid"], lab_cols=2)